In [2]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from shapely.geometry import mapping
import numpy as np
from PIL import Image
import os
import logging


In [16]:
input_geojson_path = "/Users/mischabauckhage/Documents/ETH/02_Master/3_Semester/GMP2/gmp2/01_Segmentation/output/old_national_1975_skeleton.geojson"
output_tiff_path = "/Users/mischabauckhage/Documents/ETH/02_Master/3_Semester/GMP2/gmp2/02_DEM/output/height_map_old_national_1975.png"
output_png_path = "output/"
height_attribute = "height"
input_crs = "EPSG:2056"
resolution = 0.5

In [ ]:
import numpy as np
import rasterio
from rasterio.features import rasterize
from shapely.geometry import mapping
import geopandas as gpd

# Load the GeoDataFrame
gdf = gpd.read_file(input_geojson_path)



# Select only the 'height' attribute and the geometry
gdf = gdf[['height', 'geometry']]
# Check validity of each geometry
gdf['is_valid'] = gdf['geometry'].is_valid


# Show all invalid geometries
invalid_geometries = gdf[~gdf['is_valid']]
print(invalid_geometries)

# Remove all features that have 'is_valid' == False
gdf = gdf[gdf['is_valid']]

gdf.head()




In [ ]:
#gdf = gdf.to_crs(epsg=2056)

# Verify if all geometries are now valid
print(gdf['is_valid'].all())  # Should return True if all geometries are valid


# Get the total bounds of the geometries
minx, miny, maxx, maxy = gdf.total_bounds
print(minx, miny, maxx, maxy)

gdf.head()

In [32]:

# Define the output raster dimensions (based on the resolution)

width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)

# Create an affine transform for the raster
transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, width, height)

# Create an empty array to store the raster data
raster = np.zeros((height, width), dtype=rasterio.float32)



# Prepare geometries and corresponding 'hoehe' values for rasterization
shapes = ((mapping(geom), value) for geom, value in zip(gdf.geometry, gdf[height_attribute]))

# Rasterize the geometries using the 'hoehe' values
raster = rasterize(
    shapes=shapes,
    out_shape=raster.shape,
    transform=transform,
    fill=0,  # Background value
    dtype=rasterio.float32
)

# At this point, 'raster' contains the rasterized data


In [ ]:
from PIL import Image
import cv2

def convert_tif_to_png(input_tif_path, output_png_path):
    # Load the TIF image using OpenCV
    tif_image = cv2.imread(input_tif_path, cv2.IMREAD_UNCHANGED)

    if tif_image is None:
        print(f"Failed to load TIFF file: {input_tif_path}")
        return False

    # Convert the TIF image to PNG using OpenCV
    success = cv2.imwrite(output_png_path, tif_image)

    if success:
        print(f"Successfully converted {input_tif_path} to {output_png_path}")
    else:
        print(f"Failed to convert {input_tif_path} to PNG.")

    return success

# Example usage
input_tif = 'D:\mbauckhage\gmp2\00_Transfer\dhm25.tif'
output_png = 'D:\mbauckhage\gmp2\00_Transfer\dhm25.png'
convert_tif_to_png(input_tif, output_png)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create a sample NumPy array
array = np.random.rand(10, 10).astype('float32')


# Flip the array horizontally
flipped_array = np.flipud(array)

# Plot the original and flipped arrays
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

axes[0].imshow(array, cmap='viridis')
axes[0].set_title('Original Array')

axes[1].imshow(flipped_array, cmap='viridis')
axes[1].set_title('Flipped Array')

plt.show()

# Define the file paths
original_tif_path = "original_array.tif"
flipped_tif_path = "flipped_array.tif"

# Define the transform for EPSG:2056 with bottom left pixel at (2600000, 1500000)
transform_epsg2056 = from_origin(2569000, 1183000 + array.shape[0], 1, -1)

# Ensure the arrays are 2-dimensional
array = array.squeeze()
flipped_array = flipped_array.squeeze()

import rasterio
from rasterio.transform import from_origin

# Define the file paths
original_tif_path = "original_array.tif"
flipped_tif_path = "flipped_array.tif"

# Define the transform for EPSG:2056 with bottom left pixel at (2600000, 1500000)
transform_epsg2056 = from_origin(2569000, 1183000 + array.shape[0], 1, -1)

# Ensure the arrays are 2-dimensional
array = array.squeeze()
flipped_array = flipped_array.squeeze()

# Save the original array as a TIFF file with EPSG:2056
with rasterio.open(
    original_tif_path,
    'w',
    driver='GTiff',
    height=array.shape[0],
    width=array.shape[1],
    count=1,
    dtype='float32',
    crs='EPSG:2056',
    transform=transform_epsg2056,
) as dst:
    dst.write(array, 1)

# Save the flipped array as a TIFF file with EPSG:2056
with rasterio.open(
    flipped_tif_path,
    'w',
    driver='GTiff',
    height=flipped_array.shape[0],
    width=flipped_array.shape[1],
    count=1,
    dtype='float32',
    crs='EPSG:2056',
    transform=transform_epsg2056,
) as dst:
    dst.write(flipped_array, 1)

In [ ]:


print(f"Original array saved to {original_tif_path} with EPSG:2056")
print(f"Flipped array saved to {flipped_tif_path} with EPSG:2056")


from utils.dem import create_tiles

input = "/Users/mischabauckhage/Documents/ETH/02_Master/3_Semester/GMP2/gmp2/02_DEM/flipped_array.tif"
output_png_path = "tests"

create_tiles(input, output_png_path, tile_size=2, img_format="tif",min_nonzero_value=0)

In [1]:
from utils.dem import create_tiles

input = "/Users/mischabauckhage/Documents/ETH/02_Master/3_Semester/GMP2/gmp2/02_DEM/split_merge_test/extracted_band.tif"
output_png_path = "split_merge_test/tiles"

create_tiles(input, output_png_path, tile_size=10, img_format="tif",min_nonzero_value=0, input_crs="EPSG:21781")

Processing tile (0, 0): (0, 10), (0, 10)
Tile size in meters: 25.0 x 25.0
Processing tile (1, 0): (10, 20), (0, 10)
Tile size in meters: 25.0 x 25.0
Processing tile (2, 0): (20, 28), (0, 10)
Tile size in meters: 25.0 x 25.0
Processing tile (0, 1): (0, 10), (10, 20)
Tile size in meters: 25.0 x 25.0
Processing tile (1, 1): (10, 20), (10, 20)
Tile size in meters: 25.0 x 25.0
Processing tile (2, 1): (20, 28), (10, 20)
Tile size in meters: 25.0 x 25.0
Processing tile (0, 2): (0, 10), (20, 30)
Tile size in meters: 25.0 x 25.0
Processing tile (1, 2): (10, 20), (20, 30)
Tile size in meters: 25.0 x 25.0
Processing tile (2, 2): (20, 28), (20, 30)
Tile size in meters: 25.0 x 25.0
Processing tile (0, 3): (0, 10), (30, 34)
Tile size in meters: 25.0 x 25.0
Processing tile (1, 3): (10, 20), (30, 34)
Tile size in meters: 25.0 x 25.0
Processing tile (2, 3): (20, 28), (30, 34)
Tile size in meters: 25.0 x 25.0
Tiles saved to split_merge_test/tiles


In [ ]:
import rasterio

# Define the input and output file paths
input_tif_path = input
output_tif_path = 'split_merge_test/extracted_band.tif'

# Open the input TIFF file
with rasterio.open(input_tif_path) as src:
    # Read the first band
    first_band = src.read(1)
    
    # Get the metadata of the input file
    meta = src.meta
    
    # Update the metadata to reflect the number of layers
    meta.update(count=1)

# Write the first band to a new TIFF file
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(first_band, 1)

print(f"First band extracted and saved to {output_tif_path}")